In [1]:
import pandas as pd
import numpy as np
import shutil
import sys
import os
import dlib
import json
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
is_vae = False

In [3]:
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')

In [4]:
file = 'metadataset_k_20.csv'

In [5]:
#read in the data using pandas
metadataset = pd.read_csv(os.path.join(PARSE_DATA_DIR, file ))
metadataset.head()

,original_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42,f43,f44,f45,f46,f47,f48,f49,first_place
0,1,2.092112e-32,6.249415e-32,-6.000611e-33,5.893090e-32,4.020546e-32,2.152657e-32,-5.836300e-32,2.339763e-32,4.832709e-32,...,1.450966e-33,-2.433296e-32,-1.128275e-33,-1.185880e-33,1.241357e-32,-6.068290e-33,3.252277e-32,2.221478e-32,9.671203e-34,zeroes
1,2,-1.855838e-32,-2.090192e-32,-1.898156e-32,2.504322e-32,-2.451386e-32,-6.685095e-34,-3.221332e-32,-4.198181e-32,-6.362588e-32,...,1.600629e-32,3.991929e-32,-1.531876e-32,-4.649628e-32,-4.278011e-32,2.160474e-32,-3.867348e-32,2.380331e-32,-2.335757e-32,als_ndcg
2,3,-1.169303e-32,-6.049711e-32,4.136449e-32,-9.663286e-33,-1.924273e-32,3.455974e-32,1.667681e-32,5.966751e-34,-2.579415e-32,...,5.994075e-33,-1.462464e-32,-1.664031e-32,2.628347e-33,-1.029011e-32,1.240914e-32,-1.170141e-32,-2.649629e-32,2.516531e-32,most_popular_ndcg
3,4,-2.586232e-32,-3.247477e-32,6.048788e-33,1.627761e-32,5.529978e-32,-1.838027e-33,1.555958e-32,5.057206e-32,-2.870304e-33,...,1.395353e-32,-2.553700e-32,1.501834e-32,4.594490e-32,1.732268e-32,1.482250e-32,-7.583220e-33,4.964787e-32,1.042384e-32,zeroes
4,5,-1.429949e-32,-1.369787e-32,-2.009418e-32,4.561260e-32,1.526659e-32,-2.487647e-32,5.173627e-33,-3.500297e-32,4.993383e-32,...,6.116732e-33,-1.546610e-32,-1.768405e-32,1.836809e-32,2.371812e-32,-1.883063e-32,5.819227e-32,4.135692e-32,-4.840399e-32,most_popular_ndcg


In [6]:
#als:0
#bpr:1
#lmf:2
#most_pop_3
#zeros:4
from sklearn.preprocessing import LabelEncoder
target_pre = metadataset['first_place'].values 
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target_pre)

In [7]:
target_pre

array(['zeroes', 'als_ndcg', 'most_popular_ndcg', ..., 'bpr_ndcg',
       'bpr_ndcg', 'most_popular_ndcg'], dtype=object)

In [13]:
normalize = True

In [14]:
if normalize:
  #---- SET INPUTS -----
  scaler = StandardScaler()
  #Compute the mean and std to be used for later scaling.
  scaler.fit(metadataset.drop(columns=['first_place','original_id']))
  # Perform standardization by centering and scaling
  inputs_transform = scaler.transform(metadataset.drop(columns=['first_place','original_id']))
  inputs = pd.DataFrame(inputs_transform)
  inputs.head()
else:
  inputs = metadataset.drop(columns=['first_place','original_id'])

In [15]:
metadataset_lgb = lgb.Dataset(data = inputs, 
                              label = target)


In [16]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt

# sklearn tools for model training and assesment
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score)

In [23]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': {'multi_logloss', 'auc'},
    'importance_type': 'gain',
    'num_class': 5,
    'max_bin': 255,
    'is_unbalance': False,
}

In [24]:
scoring = {'AUC':}

In [35]:
gridParams = {
    'learning_rate': np.arange(0.1, 0.2, 0.01),
    'num_leaves': np.arange(10, 100, 10),
    'colsample_bytree': np.arange(0.1, 0.5, 0.05),
    'reg_alpha': np.arange(0.05, 0.3, 0.05),
    'reg_lambda': np.arange(0.05, 0.3, 0.05),
    'max_depth':  np.arange(-1, 400, 20),
}

In [36]:
mdl = lgb.LGBMClassifier(    
    boosting_type= 'gbdt',
    objective= 'multiclass',
    metric= 'multi_logloss',
    importance_type= 'gain',
    num_class= 5,
    max_bin= 255,
    is_unbalance= False, 
    n_jobs = -1)

In [37]:
grid = GridSearchCV(mdl, gridParams, verbose=2, cv=5, scoring='accuracy', n_jobs=-1, refit='AUC')

In [38]:
grid.fit(inputs, target)

Fitting 5 folds for each of 378000 candidates, totalling 1890000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 11.3min


KeyboardInterrupt: 

In [46]:
#to tsv
inputs.to_csv(os.path.join(PARSE_DATA_DIR, 'inputs_normalized.tsv'), sep='\t', header=None, index=False)

In [49]:
df = pd.read_csv(os.path.join(PARSE_DATA_DIR, 'inputs_normalized.tsv'), sep='\t', header=None)

In [50]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
1,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
2,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
3,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
4,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
1,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
2,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
3,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418
4,-0.024315,-0.02473,-0.024062,-0.024096,-0.024189,-0.024405,-0.024405,-0.024232,-0.024043,-0.024621,...,-0.024586,-0.024683,-0.02487,-0.024779,-0.024216,-0.024449,-0.024781,-0.024835,-0.024593,-0.024418


In [53]:
pd.DataFrame(target).to_csv(os.path.join(PARSE_DATA_DIR, 'labels.tsv'), sep='\t', header=None, index=False)